In [0]:
# create a notebook for Cnn-gru prediction model 

# 1. Load the chosen dataset and try to see the vocab size 
# 2. Determine a MAX_VOCAB_SIZE incase you observe a vocab dict that is too large (choose the top/most frequent MAX_VOCAB_SIZE entries / Curse of dimensionality)
# 3. Determine a MAX_SEQUENCE LENGTH to vectorize for each review (Note: From splitter usually reviews will be limited to 400 + summary )

# DO NOT EDIT/ DELETE THIS BLOCK;



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# set working directory -> you must set the path into which you have uploaded the zipped file
# this is required in the case of colab or local 
%cd /content/drive/My\ Drive/SNLP\ Project
# %cd Source/repos/Sentiment-Analysis-using-Deep-Learning

/content/drive/My Drive/SNLP Project


In [3]:
# list content of drive - verify you are where you are supposed to be
%ls

 amz_all_beauty/                    Lexical_analysis/
 amz_all_electronics/               meta_lstm_small_200dim.tsv
 CleantData_Apr-03-2020_01-31.zip   ModelResults/
 dataset_dumps.json                 ModelVisualization/
 Electronics_5.json.gz             'Project Ideas.gdoc'
 FinalModelResults/                'Project Proposal.gdoc'
'Final Report'/                    'Report MetaData'/
 glove.6B.200d.txt                 'Sentiment Analysis - Deep Learning.pptx'
 glove.6B.zip                       vecs_lstm_small_200dim.tsv


In [4]:
# load your choice of dataset here . Specified paths as folder_datestring/file_datestring.zip
import json

with open('dataset_dumps.json') as f:
  paths_datasets = json.load(f)

print(json.dumps(paths_datasets, indent = 4))

{
    "electronics": {
        "smallTrain": "amz_all_electronics/Data_Balanced_2000_Apr-03-2020_06-46/Train_2000_Apr-03-2020_06-46.zip",
        "smallTest": "amz_all_electronics/Data_Balanced_2000_Apr-03-2020_06-46/Test_2000_Apr-03-2020_06-46.zip",
        "medTrain": "amz_all_electronics/Data_Balanced_20000_Apr-03-2020_06-52/Train_20000_Apr-03-2020_06-52.zip",
        "medTest": "amz_all_electronics/Data_Balanced_20000_Apr-03-2020_06-52/Test_20000_Apr-03-2020_06-52.zip",
        "largeTrain": "",
        "largeTest": ""
    },
    "all_beauty": {
        "smallTrain": "",
        "smallTest": "",
        "medTrain": "",
        "medTest": "",
        "largeTrain": "",
        "largeTest": ""
    }
}


In [5]:

Train_ZipCSVFileName = paths_datasets['electronics']['smallTrain']
Test_ZipCSVFileName = paths_datasets['electronics']['smallTest']

import pandas as pd
import numpy as np


df_train = pd.read_csv(Train_ZipCSVFileName)
df_train.info()

df_test = pd.read_csv(Test_ZipCSVFileName)
df_test.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   overall         10000 non-null  float64
 1   reviewTime      10000 non-null  object 
 2   reviewerID      10000 non-null  object 
 3   asin            10000 non-null  object 
 4   reviewText      10000 non-null  object 
 5   summary         10000 non-null  object 
 6   unixReviewTime  10000 non-null  int64  
 7   reviewText_len  10000 non-null  int64  
 8   summary_len     10000 non-null  int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 703.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   overall         1000 non-null   float64
 1   reviewTime      1000 non-null   object 
 2   reviewerID      1000 non-null   object 
 3  

In [6]:

# Get names of indexes for which column Age has value 30
index_neutrals_train = df_train[ df_train['overall'] == 3 ].index
index_neutrals_test = df_test[ df_test['overall'] == 3 ].index
 
# Delete these row indexes from dataFrame
df_train.drop(index_neutrals_train , inplace=True)
df_test.drop(index_neutrals_test , inplace=True)

df_train.loc[(df_train.overall == 1),'overall']= 1
df_train.loc[(df_train.overall == 2),'overall']= 1
df_train.loc[(df_train.overall == 4),'overall']= 5
df_train.loc[(df_train.overall == 5),'overall']= 5

df_test.loc[(df_test.overall == 1),'overall']= 1
df_test.loc[(df_test.overall == 2),'overall']= 1
df_test.loc[(df_test.overall == 4),'overall']= 5
df_test.loc[(df_test.overall == 5),'overall']= 5

df_train['reviewText_len'].describe()
# Since the mean average review size is around 145 chars and max is 400, I can safely set the max [summary + review] Text Limit to 400 
	

count    8000.000000
mean      146.234875
std       109.121888
min         1.000000
25%        47.000000
50%       127.000000
75%       223.250000
max       399.000000
Name: reviewText_len, dtype: float64

In [0]:
df_train['overall'].describe()
df_test['summary_len'].describe()

count    8000.000000
mean       14.786125
std         7.736512
min         2.000000
25%         9.000000
50%        11.000000
75%        20.000000
max        34.000000
Name: summary_len, dtype: float64

In [0]:
# The maximum number of words to be used. (most frequent)
MAX_VOCAB_SIZE = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 200

import uuid
folderGUID = uuid.uuid4().hex

# stupid shell way of converting variable to string 
#!mkdir "ModelResults/v3_$folderGUID"

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [9]:
tokenizer = Tokenizer(num_words= MAX_VOCAB_SIZE, filters='#$%&()*+<=>@[\\]^_`{|}~\t\n', lower=True)
tokenizer.fit_on_texts(df_train['summary'] + ' DELIM '+df_train['reviewText'])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 8218 unique tokens.


In [0]:

print(df_train['summary'].values + ' DELIM '+ df_train['reviewText'].values)

['one star DELIM volum button were dead on arriv be awar of possibl defect'
 'plug fall out DELIM work great when i tape up the plug to the port i see whi this was for sale other than that it work fine'
 'do not buy DELIM did not work im sure the seller knew this befor he ship it out date and cannot be use at all those are word from the manufactur'
 ... 'five star DELIM the number of star say it all'
 'five star DELIM good product thank'
 'five star DELIM yet anoth awesom tripp lite product i put this into a rack case with a power amplifi']


In [10]:
X = tokenizer.texts_to_sequences(df_train['summary'].values + ' DELIM '+ df_train['reviewText'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)


Shape of data tensor: (8000, 250)


In [11]:
Y = pd.get_dummies(df_train['overall']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (8000, 2)


In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout, GRU, Flatten

model = Sequential()
model.add(Embedding(MAX_VOCAB_SIZE, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(Conv1D(32,kernel_size=3,padding='same',activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.3))
model.add(Conv1D(64,kernel_size=3,padding='same',activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.35))
model.add(Conv1D(128,kernel_size=3,padding='same',activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.4))
model.add(GRU(100,return_sequences=True))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(2,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


# run for small number of epochs then save 
epochs = 12

history = model.fit(X, Y, epochs=epochs)

In [32]:
# Save the model to Local Disk 
# https://stackoverflow.com/questions/45424683/how-to-continue-training-for-a-saved-and-then-loaded-keras-model


# folderGUID = 'v3_1abe9af33fd14c8793d580bf99885182'
filePath = "ModelResults/"+str(folderGUID)+"/model.h5"
model.save(filePath)
print("Saved model to disk : "+ str(folderGUID))



Saved model to disk : 1abe9af33fd14c8793d580bf99885182


In [0]:
# Run additional training if necessary & remember to resave it 
from tensorflow.keras.models import load_model
filePath = 'ModelResults/v3_1abe9af33fd14c8793d580bf99885182/model.h5'
# Load the model
model = load_model(filePath)

# Train more on the loaded model
#history = model.fit(X, Y, epochs=5)



In [0]:


X_test = tokenizer.texts_to_sequences(df_test['summary'].values + ' DELIM '+ df_test['reviewText'].values)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_test.shape)

Y_Test = pd.get_dummies(df_test['overall']).values
print('Shape of label tensor:', Y_Test.shape)

accr = model.evaluate(X_test,Y_Test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [0]:
import matplotlib.pyplot as plt


plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.legend()
plt.show();

plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='test')
plt.legend()
plt.show();


In [40]:
from sklearn.metrics import classification_report

y_labels = df_test['overall'].values

#all precision
y_pred_softmax = model.predict(X_test)
y_pred = list()

# 1.0 for neg classes( argmax 0) and 5.0 for pos class (argmax 1)
# its just for verbosity 
for local_pred in y_pred_softmax:
  y_pred.append(1.0 if np.argmax(local_pred) == 0 else 5.0)

print(classification_report(y_labels, y_pred, target_names=['Classified Neg','Classified Pos']))



                precision    recall  f1-score   support

Classified Neg       0.90      0.95      0.93       400
Classified Pos       0.95      0.90      0.92       400

      accuracy                           0.92       800
     macro avg       0.92      0.92      0.92       800
  weighted avg       0.92      0.92      0.92       800



In [42]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 250, 200)          10000000  
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 250, 32)           19232     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 83, 32)            0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 83, 32)            0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 83, 64)            6208      
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 27, 64)            0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 27, 64)           